In [1]:
%pip install spacy




^C
Note: you may need to restart the kernel to use updated packages.


  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [556 lines of output]
      Ignoring numpy: markers 'python_version < "3.9"' don't match your environment
        Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
        Using cached cymem-2.0.10-cp313-cp313-win_amd64.whl.metadata (8.6 kB)
        Using cached preshed-3.0.9.tar.gz (14 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached murmurhash-1.0.11-cp313-cp313-win_amd64.whl.metadata (2.0 kB)
        Using cached thinc-8.

In [ ]:
import spacy
from sklearn.cluster import KMeans
from transformers import pipeline

# Load spaCy model with word vectors
nlp = spacy.load("en_core_web_md")

# Load a summarization pipeline (T5-small as an example)
summarizer = pipeline("summarization", model="t5-small")

# Input content
contents = [
    "I'm here to support your learning journey.",
    "I can help you assess your skills.",
    "Let me know your questions or concerns.",
    "How can I assist today?",
    "We can create a personalized learning path for you.",
    "I specialize in assisting with portfolio management and growth."
]

# Generate embeddings for clustering
embeddings = [nlp(content).vector for content in contents]

# Perform KMeans clustering
num_clusters = 3  # Define the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(embeddings)

# Summarize cluster content
cluster_intents = {}
for cluster in range(num_clusters):
    # Get all sentences in this cluster
    cluster_texts = [contents[i] for i in range(len(contents)) if labels[i] == cluster]
    combined_text = " ".join(cluster_texts)  # Combine sentences into a single block
    
    # Summarize the combined text
    summarized = summarizer(combined_text, max_length=20, min_length=5, do_sample=False)
    intent = summarized[0]["summary_text"]
    
    # Store the intent
    cluster_intents[cluster] = intent

# Output results
print("\n### Clustered Content and Generated Intents ###\n")
for cluster, intent in cluster_intents.items():
    print(f"Cluster {cluster} Intent: {intent}")
    print(f"Cluster {cluster} Contents:")
    cluster_texts = [contents[i] for i in range(len(contents)) if labels[i] == cluster]
    for text in cluster_texts:
        print(f"  - {text}")
    print()


In [ ]:
%pip install rake-nltk


In [ ]:
%pip install scikit-learn


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [8]:
from sklearn.cluster import KMeans
from rake_nltk import Rake
import numpy as np

# Input content
contents = [
    "Impressive, Cosmo! Managing Director is a pivotal role. Could you share which industry you're leading in? 🏢 Also, how many years of experience do you have in this field? This will help us tailor your learning journey even further. 🌟",
"Thank you for sharing that, Cosmo! Taking a new direction in life can be really empowering. 💪 Let's keep moving forward. Could you tell me about your current role, if you're working, and the industry you're in? It’ll help us create the most relevant and engaging learning path for you! 😊"
]

# Initialize RAKE for keyword extraction
rake = Rake()

# Function to extract keywords using RAKE
def extract_keywords(text):
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()

# Step 1: Extract keywords from each sentence
keywords = [extract_keywords(content) for content in contents]

# Step 2: Vectorize keyword lists to cluster them
# We'll use the length of the keyword list as a simple feature representation
keyword_vectors = np.array([len(k) for k in keywords]).reshape(-1, 1)

# Perform clustering with KMeans
num_clusters = 1  # Define the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(keyword_vectors)

# Step 3: Group content by clusters and assign intent based on top keywords
cluster_intents = {}
for cluster in range(num_clusters):
    # Get all sentences in this cluster
    cluster_texts = [contents[i] for i in range(len(contents)) if labels[i] == cluster]
    combined_text = " ".join(cluster_texts)  # Combine sentences into a single block
    
    # Extract keywords for the combined text of the cluster
    intent_keywords = extract_keywords(combined_text)
    # Get the top-ranked keyword (for simplicity, use the first one)
    intent = intent_keywords[0] if intent_keywords else "Unknown intent"
    
    # Store the intent
    cluster_intents[cluster] = intent

# Output results
print("\n### Clustered Content and Generated Intents ###\n")
for cluster, intent in cluster_intents.items():
    print(f"Cluster {cluster} Intent: {intent}")
    print(f"Cluster {cluster} Contents:")
    cluster_texts = [contents[i] for i in range(len(contents)) if labels[i] == cluster]
    for text in cluster_texts:
        print(f"  - {text}")
    print()



### Clustered Content and Generated Intents ###

Cluster 0 Intent: learning journey even
Cluster 0 Contents:
  - Impressive, Cosmo! Managing Director is a pivotal role. Could you share which industry you're leading in? 🏢 Also, how many years of experience do you have in this field? This will help us tailor your learning journey even further. 🌟
  - Thank you for sharing that, Cosmo! Taking a new direction in life can be really empowering. 💪 Let's keep moving forward. Could you tell me about your current role, if you're working, and the industry you're in? It’ll help us create the most relevant and engaging learning path for you! 😊



In [10]:
import re

# Lists of keywords for each stage
profile = ["first name", "last name", "age", "education level", "current role", "industry", "years of experience", "department", "manager", "job duration"]
discover = ["interests", "goals", "learning style", "knowledge level", "application area", "rationale"]
discuss = ["specific interest areas", "learning style", "available study time per week", "timeline"]

# Input stage and content from the user
stage = input("Enter the stage (profile, discover, discuss): ").lower().strip()
content = input("Enter the content: ").lower().strip()

# Dictionary to map stage to corresponding keyword list
stage_keywords = {
    'profile': profile,
    'discover': discover,
    'discuss': discuss
}

# Function to match content with keywords and identify multiple matches
def match_keywords(content, keywords):
    # Find all keyword matches in the content using regex
    matched_keywords = [keyword for keyword in keywords if re.search(r'\b' + re.escape(keyword) + r'\b', content)]
    
    # Return matched keywords (if any)
    return matched_keywords

# Check if the input stage is valid
if stage in stage_keywords:
    # Get the keyword list for the given stage
    keywords = stage_keywords[stage]
    
    # Apply keyword matching for the content
    matched_keywords = match_keywords(content, keywords)
    
    # Output results
    if matched_keywords:
        print("\n### Matched Keywords ###")
        print(f"Content matched with the stage '{stage}':")
        print(f"Intent: {', '.join(matched_keywords)}")
    else:
        print(f"\nNo matches found for the given content in the '{stage}' stage.")
else:
    print("Invalid stage entered. Please choose from 'profile', 'discover', or 'discuss'.")



### Matched Keywords ###
Content matched with the stage 'profile':
Intent: current role, industry
